In [1]:
from STK_Sim import *

Filename = 'AERO_401_Initial_Assessment'

stk_object = STK_Simulation(False,Filename)

In [51]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
dt = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+dt).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

In [54]:
# Generating a polygon to bound the lat/lon coordinates, you can create your polygon, in the same format as Targets_Polygon.txt.
poly = Create_Poly('Input_Files/Targets_Polygon.txt')
# Writing random points within the polygon to a target file.
targets_filename = 'Input_Files/Targets_File.txt'
polygon_random_points(poly,1).to_csv(targets_filename,index=False)
# Plotting the polygon and generated targets on the map.
plot_targets_and_polygon(poly,targets_filename)
# Loading targets into stk from file.
stk_object.Target_Loader(targets_filename)

In [ ]:
# Generating an external sensor pointing file (still in development, feel free to comment out)
External_Pointing_File = '/Users/jjfoo/Spring_2024_Classes/AERO_401/Input_Files/External_Pointing_File.txt'
Pointing_File_Generator(External_Pointing_File,2*86400)

In [55]:
# Loading satellites into stk from file.
satellites_filename = 'Input_Files/Satellites_File.txt'
stk_object.Satellite_Loader(satellites_filename)

# Loading satellites into stk from file with external pointing file (Comment out the one you do not want to use).
# satellites_filename = 'Input_Files/Satellites_File.txt'
# stk_object.Satellite_Loader(satellites_filename,External_Pointing_File)

In [ ]:
# Calculating the lifetime of each satellite using satellite properties
Cd=2.2
Cr=1.0
DragArea=13.65
SunArea=15.43
Mass=100.0
stk_object.Compute_Lifetime(Cd,Cr,DragArea,SunArea,Mass)
print(stk_object.Lifetimes)

In [ ]:
# Computing Azimuth and Elevation angels with a given time value (dt in seconds) in between each Computation.
dt = 60
stk_object.Compute_AzEl(dt)

In [43]:
# Visualization of Targets Azimuth and Elevation data (includes all satellites).

figs1 = []
figs2 = []
buttons1 = []
buttons2 = []

fig3 = go.Figure()

for t in range(len(stk_object.targets)):
    fig1 = px.imshow(stk_object.Azimuth_vs_Elevation[f"Target{t+1}"].transpose(), text_auto=True,
                labels=dict(x="Azimuth",y="Elevation", color="Total Frames Captured"),)
    figs1.append(fig1.data[0])
    for s in range(len(stk_object.sensors)):
        df = stk_object.AzEl_data[f'Target{t+1}->Satellite{s+1}']
        if type(df) != int:
            df['Azimuth'] = df['Azimuth'].astype(float)
            df['Elevation'] = df['Elevation'].astype(float)
            fig2 = go.Scatter(y=df['Elevation'],x=df['Azimuth'],name=f'{t+1}->{s+1}',mode='lines')
            figs2.append(fig2)
        else:
            figs2.append(go.Scatter(x=[],y=[]))

    if type(stk_object.time_sorted_data[f'Target{t+1}']) != int:
        df = pd.DataFrame(stk_object.time_sorted_data[f'Target{t+1}'])
        fig3.add_trace(go.Scatter(x=df['Time'],y=df['Percent Imaged'],name=f'Target{t+1}'))

    z1 = np.zeros(len(stk_object.targets),dtype=bool);z1[t]=1
    z2 = np.zeros(len(stk_object.targets)*len(stk_object.sensors),dtype=bool)
    z2[len(stk_object.sensors)*t] = 1
    for s in range(len(stk_object.sensors)):
        z2[len(stk_object.sensors)*t+s]=1
    buttons1.append(dict(label=f"Target{t+1}",
                        method="restyle",
                        args=[{"visible": z1},]))
    buttons2.append(dict(label=f"Target{t+1}",
                        method="restyle",
                        args=[{"visible": z2},]))

fig1=go.Figure(figs1)
fig1.update_layout(xaxis_title="Azimuth", yaxis_title="Elevation",
    updatemenus=[
        dict(
            active=0,
            buttons=buttons1,
              x=1.1,
              xanchor="left",
              y=1.1,
              yanchor="top")])

fig2=go.Figure(figs2)
fig2.update_layout(xaxis_title="Azimuth", yaxis_title="Elevation",
    updatemenus=[
        dict(
            active=0,
            buttons=buttons2,
              x=1.1,
              xanchor="left",
              y=1.1,
              yanchor="top")])

fig3.update_layout(xaxis_title="Time", yaxis_title="% Imaged")

fig1.show()
fig2.show()
fig3.show()

# f1 = PlotlyViewer(fig1)
# f2 = PlotlyViewer(fig2)
# f3 = PlotlyViewer(fig3)

In [ ]:
num_total_angles = 0
for t in range(len(stk_object.targets)):
    num_total_angles += len(np.where(stk_object.Azimuth_vs_Elevation[f"Target{t+1}"]>0)[0])
print(f"num_total_angles = {100*num_total_angles/162/len(stk_object.targets)}%")

In [34]:
dt = 60
stk_object.Compute_YPR_rates(dt)

In [46]:
figs = []
buttons = []

max_yaw_rate = 0
max_pitch_rate = 0
max_roll_rate = 0

for s in range(len(stk_object.sensors)):
    df = stk_object.YPR_rates_data[f"Satellite{s+1}"]
    if max(abs(df['yaw rate'])) > max_yaw_rate:
        max_yaw_rate = max(abs(df['yaw rate']))
    if max(abs(df['pitch rate'])) > max_pitch_rate:
        max_pitch_rate = max(abs(df['pitch rate']))
    if max(abs(df['roll rate'])) > max_roll_rate:
        max_roll_rate = max(abs(df['roll rate']))
    fig = px.line(df, x='Time', y=['yaw rate', 'pitch rate', 'roll rate'],markers=True)
    figs.append(fig.data[0])
    figs.append(fig.data[1])
    figs.append(fig.data[2])
    z = np.zeros(3*len(stk_object.sensors),dtype=bool);z[s]=1;z[s+1]=1;z[s+2]=1
    buttons.append(dict(label=f"Sensor{s+1}",
                        method="restyle",
                        args=[{"visible": z},]))

fig=go.Figure(figs)
fig.update_layout(xaxis_title="Time (s)", yaxis_title="rate (deg/s)",
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
              x=1.1,
              xanchor="center",
              y=1.1,
              yanchor="top")])
fig.show()

In [ ]:
print("max_yaw_rate =",max_yaw_rate)
print("max_pitch_rate =",max_pitch_rate)
print("max_roll_rate =",max_roll_rate)